<a href="https://colab.research.google.com/github/DhaneshGavimath/NLP-Projects/blob/main/model/chatbot%20modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# pip install jupyterthemes
# !jt -l

In [3]:
# Choosing the notebook theme
# !jt -t monokai

Imports

In [4]:
import pandas as pd
import numpy as np
import json
import nltk 
import sklearn

In [5]:
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from string import punctuation

In [14]:
nltk.download("stopwords")
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
"""
classes
"""
intents_classes = ["greetings", "goodbye","fee_structure", "campus_placement", "departments", 
                    "adress","activities", "commute","gratitude"]
intents_classes.sort()
classes_code_label = dict(map(lambda x:(intents_classes.index(x),x),intents_classes))
classes_label_code = dict(map(lambda x:(x,intents_classes.index(x)),intents_classes))
# print("-------------classes_str_to_num-----------\n",classes_str_to_num)
# print("-------------classes_num_to_str-----------\n",classes_num_to_str)

### Load Data
 Here we will be loading the json we made containing the questions user might ask.
 as we run this file it show load the training file, perform datacleaning, vector repressentation
 and finally creating a dataframe and write to current folder. 
 we will be running this file only to retrain the model with updated data we might get new words.
    

In [ ]:
def load_chat_data():
    # loading the data  
    with open("train_classes.json",'r') as jdata:
        data = json.load(jdata)

    train_data = []

    for intent, sentences in data.items():
        input_target_pair = list(map(lambda x: (x,intent),sentences))
        train_data.extend(input_target_pair)

    train_dataframe = pd.DataFrame(train_data, columns = ["input","target"])
    return train_dataframe

data = load_chat_data()
data.head()

In [10]:
"""
Replacing the target with numeric value
"""
data["target"] = data["target"].map(classes_label_code)

In [12]:
def tokenize(data):
    # Word tokenize
    tokens = word_tokenize(data)
    return tokens

def remove_junks(tokens):
    # Removing punctuations
    wo_punctuations  = set(tokens).difference(set(punctuation))
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    wo_stopwords_puncts = wo_punctuations.difference(stop_words)
    return list(wo_stopwords_puncts)

def stemming(clean_words):
    stem_obj = PorterStemmer()
    stem_words = list(map(lambda x:stem_obj.stem(x),clean_words))
    return stem_words

def create_vocabulary(data, column):
    # creating a vocanulary out of text column
    vocab = []
    for record in data[column].values:
      vocab.extend(record)
    return set(vocab)

def clean_data(df, column):
    df[column]= df[column].apply(tokenize)
    df[column]= df[column].apply(remove_junks)
    df[column]= df[column].apply(stemming)
    return df

In [15]:
cleaned_data = clean_data(data,"input")

In [18]:
# cleaned_data